# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_file=pd.read_csv('../output_data/weather_data_df.csv')
droped_df=input_file.loc[:, ~input_file.columns.str.contains('^Unnamed')]
# city=input_file.loc[input_file['Country']=='VN']
droped_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,12,PF,1593530599,62,-23.12,-134.97,68.70,17.74
1,Esperance,75,AU,1593530947,76,-33.87,121.90,55.40,31.09
2,Port Alfred,25,ZA,1593530949,82,-33.59,26.89,60.01,7.99
3,Rochegda,100,RU,1593530950,42,62.80,43.25,64.17,9.86
4,Bonfim,99,BR,1593530951,80,3.08,-59.95,79.93,2.68
...,...,...,...,...,...,...,...,...,...
576,Rhyl,100,GB,1593531695,71,53.32,-3.49,62.01,1.01
577,Chulumani,2,BO,1593531697,15,-16.41,-67.53,81.77,3.40
578,Khovd,24,MN,1593531698,50,48.01,91.64,62.46,6.31
579,Tamasane,0,BW,1593531699,53,-22.42,27.40,71.60,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Convert the humidity to float and drop null cells
input_file=input_file.dropna()
locations=input_file[['Lat','Lng']].astype(float)
hum_weights=input_file['Humidity'].astype(float)


In [15]:
# Create humidity heatmap layers
fig=gmaps.figure()
figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'
}
heat_layer=gmaps.heatmap_layer(locations,weights=hum_weights,
                               dissipating=False,max_intensity=100,
                               point_radius=1)
fig=gmaps.figure(map_type='HYBRID',layout=figure_layout)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='400px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
cleaned_cities_df=droped_df
cleaned_cities_df=cleaned_cities_df.loc[(cleaned_cities_df['Max Temp']>70) & (cleaned_cities_df['Max Temp']<80)]
cleaned_cities_df=cleaned_cities_df.loc[cleaned_cities_df['Wind Speed']<10]
cleaned_cities_df=cleaned_cities_df.loc[cleaned_cities_df['Cloudiness']==0]
cleaned_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
34,Kumul,0,CN,1593530841,19,42.80,93.45,78.71,7.05
80,Senanga,0,ZM,1593531049,21,-16.12,23.27,73.44,6.11
96,Lüderitz,0,NaN,1593531070,24,-26.65,15.16,78.80,4.70
125,Mayumba,0,GA,1593531108,75,-3.43,10.66,74.30,7.92
142,Bengkulu,0,ID,1593531130,76,-3.80,102.27,76.51,4.85
144,Dingzhou,0,CN,1593531134,88,38.51,115.00,71.60,2.24
188,Javorník,0,CZ,1593531191,39,50.39,17.00,73.00,3.00
233,Hernani,0,ES,1593531249,73,43.27,-1.98,77.00,8.05
240,Yumen,0,CN,1593531259,29,40.28,97.20,71.51,8.23
273,Udachny,0,RU,1593531301,29,66.42,112.40,75.20,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df=cleaned_cities_df
hotel_names=[]
hotel_df['Hotel Name']=''
# Create the base url 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# Create the parameters for 5000 meters
params = {
    "radius":5000,
    "type": "hotel",
    "key": g_key,
}


In [20]:
# Use iterrows to loop through the df
for index,row in hotel_df.iterrows():
    coordinates=f"{row['Lat']},{row['Lng']}"
    params['location']=coordinates
    get_response=requests.get(base_url,params=params).json()
    hotel_names.append(get_response['results'][1]['name'])
    try:
        hotel_df.loc[index, "Hotel Name"]=get_response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Skipped location.")

In [21]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
34,Kumul,0,CN,1593530841,19,42.80,93.45,78.71,7.05,Hami
80,Senanga,0,ZM,1593531049,21,-16.12,23.27,73.44,6.11,Senanga
96,Lüderitz,0,NaN,1593531070,24,-26.65,15.16,78.80,4.70,Luderitz
125,Mayumba,0,GA,1593531108,75,-3.43,10.66,74.30,7.92,Mayumba
142,Bengkulu,0,ID,1593531130,76,-3.80,102.27,76.51,4.85,Bengkulu
144,Dingzhou,0,CN,1593531134,88,38.51,115.00,71.60,2.24,Baoding
188,Javorník,0,CZ,1593531191,39,50.39,17.00,73.00,3.00,Javorník
233,Hernani,0,ES,1593531249,73,43.27,-1.98,77.00,8.05,Donostia-San Sebastian
240,Yumen,0,CN,1593531259,29,40.28,97.20,71.51,8.23,Jiuquan
273,Udachny,0,RU,1593531301,29,66.42,112.40,75.20,4.47,Udachny


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
marker_layers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='400px'))